In [96]:
import requests
import pandas as pd
import urllib3
# import json

In [97]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [98]:
url_atual = "https://economia.awesomeapi.com.br/json/last/USD-BRL,EUR-BRL,BTC-BRL,GBP-BRL"

In [99]:
response1 = requests.get(url_atual, verify=False,timeout=5)
dados_atual = response1.json()


In [100]:
df_atual = pd.DataFrame(dados_atual).T
df_atual = df_atual[['code', 'name', 'bid', 'ask', 'pctChange']].copy()
df_atual.columns = ['Codigo', 'Moeda', 'Compra_Atual', 'Venda_Atual', 'variacao' ]

In [101]:
df_atual['Compra_Atual'] = pd.to_numeric(df_atual['Compra_Atual'])
df_atual['Venda_Atual'] = pd.to_numeric(df_atual['Venda_Atual'])
df_atual['variacao'] = pd.to_numeric(df_atual['variacao'])

In [102]:
print(f'Cotacao Atual {df_atual}')

Cotacao Atual        Codigo                            Moeda  Compra_Atual   Venda_Atual  \
USDBRL    USD  Dólar Americano/Real Brasileiro       5.38950       5.39250   
EURBRL    EUR             Euro/Real Brasileiro       6.31712       6.33312   
BTCBRL    BTC          Bitcoin/Real Brasileiro  498001.00000  498002.00000   
GBPBRL    GBP  Libra Esterlina/Real Brasileiro       7.21156       7.21557   

        variacao  
USDBRL -0.214219  
EURBRL -0.378966  
BTCBRL  1.706000  
GBPBRL -0.192030  


In [103]:
url_historico = "https://economia.awesomeapi.com.br/json/daily/USD-BRL/30"
response2 = requests.get(url_historico, verify=False, timeout=10)
dados_historico = response2.json()

In [104]:
df_historico = pd.DataFrame(dados_historico)
df_historico = df_historico[['code', 'timestamp', 'bid', 'ask']].copy()
df_historico.columns = ['Codigo', 'Data', 'Compra_Historica', 'Venda_Historica']

In [105]:
df_historico['Data'] = pd.to_datetime(pd.to_numeric(df_historico['Data']), unit='s')
df_historico['Compra_Historica'] = pd.to_numeric(df_historico['Compra_Historica'])
df_historico['Venda_Historica'] = pd.to_numeric(df_historico['Venda_Historica'])

In [106]:
df_historico_30d = df_historico.iloc[-1:].copy()

In [107]:
df_completo = pd.merge(df_atual, df_historico_30d, on='Codigo', how='left')

In [108]:
df_completo['Variacao_30d'] = (
    (df_completo['Compra_Atual'] - df_completo['Compra_Historica']) /
    df_completo['Compra_Historica'] * 100
).round(2)
# Calcular Variação de 30 Dias  //  ((Valor Atual - Valor Antigo) / Valor Antigo) × 100

In [109]:
print(df_completo)

  Codigo                            Moeda  Compra_Atual   Venda_Atual  \
0    USD  Dólar Americano/Real Brasileiro       5.38950       5.39250   
1    EUR             Euro/Real Brasileiro       6.31712       6.33312   
2    BTC          Bitcoin/Real Brasileiro  498001.00000  498002.00000   
3    GBP  Libra Esterlina/Real Brasileiro       7.21156       7.21557   

   variacao Data  Compra_Historica  Venda_Historica  Variacao_30d  
0 -0.214219  NaT               NaN              NaN           NaN  
1 -0.378966  NaT               NaN              NaN           NaN  
2  1.706000  NaT               NaN              NaN           NaN  
3 -0.192030  NaT               NaN              NaN           NaN  


In [110]:

# with open('carteira_multipla.ipynb', 'r', encoding='utf-8') as f:
#     notebook = json.load(f)
#
# with open('carteira_multipla_completo.txt', 'w', encoding='utf-8') as output:
#     for i, cell in enumerate(notebook['cells'], 1):
#         output.write(f"\n{'='*60}\n")
#         output.write(f"CÉLULA {i} - TIPO: {cell['cell_type'].upper()}\n")
#         output.write(f"{'='*60}\n\n")
#
#         # Conteúdo da célula
#         output.write(''.join(cell['source']))
#         output.write('\n\n')
#
#         # Se tiver outputs (resultados)
#         if 'outputs' in cell and cell['outputs']:
#             output.write("\n--- SAÍDA ---\n")
#             for out in cell['outputs']:
#                 if 'text' in out:
#                     output.write(''.join(out['text']))
#                 elif 'data' in out and 'text/plain' in out['data']:
#                     output.write(''.join(out['data']['text/plain']))
#             output.write("\n")
#
# print("Conversão concluída!")


#apenas conversão de arquivos